In [1]:
import pandas as pd 
import datetime as dt
df = pd.read_csv('online.csv')
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom


In [2]:
df.shape

(70864, 9)

In [3]:
df.describe()

,Unnamed: 0,InvoiceNo,Quantity,UnitPrice,CustomerID
count,70864.000000,70864.000000,70864.000000,70864.000000,70864.00000
mean,278797.962590,560639.843136,11.511233,3.063033,15552.19790
std,153606.252964,13176.494003,41.017582,31.894970,1595.50593
min,3.000000,536365.000000,1.000000,0.001000,12747.00000
25%,147667.250000,549128.000000,2.000000,1.250000,14194.00000
50%,286056.000000,562012.500000,4.000000,1.950000,15525.00000
75%,412258.750000,572283.000000,12.000000,3.750000,16931.00000
max,541893.000000,581586.000000,4300.000000,8142.750000,18287.00000


In [4]:
df.isna().sum()

Unnamed: 0     0
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70864 entries, 0 to 70863
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   70864 non-null  int64  
 1   InvoiceNo    70864 non-null  int64  
 2   StockCode    70864 non-null  object 
 3   Description  70864 non-null  object 
 4   Quantity     70864 non-null  int64  
 5   InvoiceDate  70864 non-null  object 
 6   UnitPrice    70864 non-null  float64
 7   CustomerID   70864 non-null  int64  
 8   Country      70864 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 4.9+ MB


In [6]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [7]:
df['total_price'] = df['Quantity'] * df['UnitPrice']

In [8]:
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_price
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom,12.60
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom,1.45
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom,22.50
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom,2.10
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom,17.85


In [9]:
min(df['InvoiceDate'])

Timestamp('2010-12-01 08:26:00')

In [10]:
max(df['InvoiceDate'])

Timestamp('2011-12-09 12:49:00')

In [11]:
earliest = max(df['InvoiceDate']) + dt.timedelta(days = 1)

In [12]:
df.rename(columns={'InvoiceDate': 'Recency','InvoiceNo': 'Frequency','total_price': 'Monetary'}, inplace=True)

In [13]:
rfm_pivoted = df.groupby(['CustomerID']).agg({
'Recency': lambda last_invoice_date : (earliest - last_invoice_date.max()).days ,
'Frequency': 'count' , 'Monetary' : 'sum'})

In [14]:
rfm_pivoted.head()

,Recency,Frequency,Monetary
CustomerID,,,
12747,2,27,992.82
12748,1,967,7522.06
12749,4,37,813.45
12820,3,17,268.02
12822,71,9,146.15


we will segment customers into four quartiles and assign label for each quartile

In [15]:
r_labels = range(4,0,-1)
f_labels = range(1,5)
m_labels = range(1,5)
r_quartile = pd.qcut(rfm_pivoted['Recency'] , 4 , labels = r_labels)
f_quartile = pd.qcut(rfm_pivoted['Frequency'] , 4 , labels = f_labels)
m_quartile = pd.qcut(rfm_pivoted['Monetary'] , 4 , labels = m_labels)
rfm_pivoted = rfm_pivoted.assign(r = r_quartile.values)
rfm_pivoted = rfm_pivoted.assign(f = f_quartile.values)
rfm_pivoted = rfm_pivoted.assign(m = m_quartile.values)

In [16]:
rfm_pivoted.sample(10)

,Recency,Frequency,Monetary,r,f,m
CustomerID,,,,,,
12853,135,12,234.48,2,3,3
18251,87,3,444.96,2,1,4
12976,212,5,203.68,1,2,3
12999,205,2,21.90,1,1,1
15986,30,2,30.00,3,1,1
13314,2,11,133.71,4,3,2
16983,13,30,236.32,4,4,3
14353,215,2,18.90,1,1,1
17562,51,12,204.36,3,3,3


In [17]:
rfm_pivoted['segment'] = rfm_pivoted['r'].astype(str) + rfm_pivoted['f'].astype(str)  + rfm_pivoted['m'].astype(str)
rfm_pivoted['score'] = rfm_pivoted[['r','f','m']].sum(axis=1)

In [18]:
rfm_pivoted.head()

,Recency,Frequency,Monetary,r,f,m,segment,score
CustomerID,,,,,,,,
12747,2,27,992.82,4,4,4,444,12
12748,1,967,7522.06,4,4,4,444,12
12749,4,37,813.45,4,4,4,444,12
12820,3,17,268.02,4,3,3,433,10
12822,71,9,146.15,2,2,3,223,7


In [19]:
rfm_pivoted.groupby('segment').size().shape

(62,)

In [20]:
rfm_pivoted.groupby('segment').size().sort_values(ascending=False)[:5]

segment
444    382
111    346
211    168
344    162
233    132
dtype: int64

# Summary metrics per RFM score

In [21]:
rfm_pivoted.groupby('score').agg({'Recency': 'mean','Frequency': 'mean','Monetary': ['mean', 'count'] }).astype(int)

Recency Frequency Monetary      
         mean      mean     mean count
score                                 
3         256         2       28   346
4         175         3       47   348
5         145         4       79   406
6         105         6      149   433
7          82         8      163   384
8          63        12      198   382
9          48        16      330   347
10         32        24      438   364
11         21        38      709   309
12          7        77     1709   382

# Grouping into named segments

In [25]:
def segment_me(data_frame):
    if data_frame['score'] >= 9:
        return 'can not lose'
    elif (data_frame['score'] >= 5) and (data_frame['score'] < 9):
        return 'need attention'
    else:
        return 'rebuild loyality'
rfm_pivoted['General_Segment'] = rfm_pivoted.apply(segment_me, axis=1)
rfm_pivoted.groupby('General_Segment').agg({
'Recency': 'mean',
'Frequency': 'mean',
'Monetary': 'mean'}).astype(int)

,Recency,Frequency,Monetary
General_Segment,,,
can not lose,27,40,817
need attention,100,7,146
rebuild loyality,216,2,38
